In [1]:
from pytorch_lightning import Trainer
from catinous.CatsinomModelGramCache import CatsinomModelGramCache
import catinous.CatsinomModelGramCache as catsmodel
from catinous import utils as utils
from catinous.CatsinomDataset import CatsinomDataset

import matplotlib.pyplot as plt
from pytorch_lightning.callbacks import ModelCheckpoint
from torch.utils.data import DataLoader
import os
import torchvision.models as models
import torch.nn as nn
import torch
import torch.nn.functional as F
import argparse
import pytorch_lightning as pl
import sklearn 
from sklearn.metrics import confusion_matrix, auc, roc_curve
import torch
import pandas as pd
import seaborn as sns
import pickle
from py_jotools import mut, slurm
import numpy as np
import gc
import hashlib
import dill

%load_ext autoreload
%autoreload 

In [10]:
# tensor([[ 0.0072,  0.0071, -0.0101,  ...,  0.0065,  0.0051,  0.0175],
#         [-0.0102,  0.0059,  0.0228,  ..., -0.0115,  0.0006, -0.0125],
#         [ 0.0167, -0.0050,  0.0069,  ..., -0.0075, -0.0214,  0.0203],
#         ...,
#         [-0.0205,  0.0164, -0.0105,  ..., -0.0155,  0.0018, -0.0116],
#         [-0.0189,  0.0157,  0.0153,  ..., -0.0129, -0.0067, -0.0168],
#         [-0.0019,  0.0022, -0.0098,  ...,  0.0199,  0.0004,  0.0170]],

In [2]:
hparams={'continous':False,
         'datasetfile': 'catsinom_lr_dataset.csv',
         'noncontinous_train_splits': ['base_train'],
         'noncontinous_steps': 3000}
blubb, logs, df_cache, basemodel_lr = catsmodel.trained_model(hparams)

Read: /project/catinous/trained_models/batch_lr_base_train_1_2d20289ac9.pt


In [4]:
blubb.model.fc[0].weight

Parameter containing:
tensor([[ 0.0072,  0.0071, -0.0101,  ...,  0.0065,  0.0051,  0.0175],
        [-0.0102,  0.0059,  0.0228,  ..., -0.0115,  0.0006, -0.0125],
        [ 0.0167, -0.0050,  0.0069,  ..., -0.0075, -0.0214,  0.0203],
        ...,
        [-0.0205,  0.0164, -0.0105,  ..., -0.0155,  0.0018, -0.0116],
        [-0.0189,  0.0157,  0.0153,  ..., -0.0129, -0.0067, -0.0168],
        [-0.0019,  0.0022, -0.0098,  ...,  0.0199,  0.0004,  0.0170]])

In [5]:
hparams={'continous': True,
         'use_cache': False,
         'datasetfile': 'catsinom_combined_dataset.csv',
         'base_model': basemodel_lr,
         'EWC': False,
         'EWC_dataset': 'catsinom_lr_dataset.csv',
         'EWC_lambda': 10000000000000,
         'val_check_interval': 100}
# model, logs, df_cache, basemodel_lr = catsmodel.trained_model(hparams, show_progress = True)

In [74]:
dl = DataLoader(CatsinomDataset(model.hparams.root_dir,
                                    'catsinom_combined_dataset.csv',
                                    split='val'),
                    batch_size=4, num_workers=2)

In [75]:
x,y,img,res = next(iter(dl))

In [6]:
def do_the_eval(model):
    dl = DataLoader(CatsinomDataset(model.hparams.root_dir,
                                    'catsinom_combined_dataset.csv',
                                    split='val'),
                    batch_size=4, num_workers=2)
    
    val_acc_lr_l = []
    val_acc_hr_l = []
    model.freeze()
    with torch.no_grad():
        for x, y, img, res in dl:

            y_hat = model.forward(x.float().cuda())
            y_sig = torch.sigmoid(y_hat)
            y_sig = (y_sig > torch.Tensor([0.5]).cuda()).long()

            if res[0] == 'lr':
                val_acc_lr_l.extend((y[:,None].cuda()==y_sig).detach().cpu().numpy())
            else:
                val_acc_hr_l.extend((y[:,None].cuda()==y_sig).detach().cpu().numpy())

    print('HR: %f' % (np.asarray(val_acc_hr_l).sum()/len(val_acc_hr_l)))
    print('LR: %f' % (np.asarray(val_acc_lr_l).sum()/len(val_acc_lr_l)))                         


In [15]:
do_the_eval(blubb.cuda())

HR: 0.565321
LR: 0.913158


In [8]:
# from torch.utils.data import DataLoader
# from catinous.CatsinomDataset import CatsinomDataset
# dl = DataLoader(CatsinomDataset(model.hparams.root_dir,
#                                               model.hparams.datasetfile,
#                                               iterations=100,
#                                               batch_size=8,
#                                               split=model.hparams.noncontinous_train_splits),
#                               batch_size=8, num_workers=2)

In [9]:
model = CatsinomModelGramCache(hparams=hparams, device=torch.device('cuda'))

# model.unfreeze()
# model.cuda()
# cm = utils.EWC(model.model, dl)

In [17]:
model.loss

BCEWithLogitsLoss()

In [112]:
do_the_eval(model.cuda())

HR: 0.888361
LR: 0.528947


In [113]:
do_the_eval(blubb.cuda())

HR: 0.565321
LR: 0.913158


In [19]:
model.unfreeze()

In [12]:
logger = utils.pllogger(model.hparams)
trainer = Trainer(gpus=1, max_epochs=1, early_stop_callback=False, logger=logger, val_check_interval=model.hparams.val_check_interval, show_progress_bar=True, checkpoint_callback=False)
trainer.fit(model)

INFO:root:GPU available: True, used: True
INFO:root:VISIBLE GPUS: 0
INFO:root:
    | Name                        | Type              | Params
--------------------------------------------------------------
0   | model                       | ResNet            | 24 M  
1   | model.conv1                 | Conv2d            | 9 K   
2   | model.bn1                   | BatchNorm2d       | 128   
3   | model.relu                  | ReLU              | 0     
4   | model.maxpool               | MaxPool2d         | 0     
5   | model.layer1                | Sequential        | 215 K 
6   | model.layer1.0              | Bottleneck        | 75 K  
7   | model.layer1.0.conv1        | Conv2d            | 4 K   
8   | model.layer1.0.bn1          | BatchNorm2d       | 128   
9   | model.layer1.0.conv2        | Conv2d            | 36 K  
10  | model.layer1.0.bn2          | BatchNorm2d       | 128   
11  | model.layer1.0.conv3        | Conv2d            | 16 K  
12  | model.layer1.0.bn3          | Bat

KeyboardInterrupt: 

In [35]:
model.freeze()

In [36]:
blubb.model.bn1.weight[0]

tensor(0.2392, device='cuda:0')

In [105]:
for n,p in model.model.named_parameters():
#     print(n)
    print(torch.sum(blubb.model.state_dict()[n] - p) == 0)

tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(True, device='cuda:0')
tensor(Tru

In [68]:
torch.sum(blubb.model.state_dict()[n] - p)

tensor(0., device='cuda:0')

In [66]:
p.shape

torch.Size([64, 3, 7, 7])

In [37]:
model.model.bn1.weight[0]

tensor(0.2392, device='cuda:0')

In [38]:
model.model.fc[0].weight

Parameter containing:
tensor([[ 0.0072,  0.0071, -0.0101,  ...,  0.0065,  0.0051,  0.0175],
        [-0.0102,  0.0059,  0.0228,  ..., -0.0115,  0.0006, -0.0125],
        [ 0.0167, -0.0050,  0.0069,  ..., -0.0075, -0.0214,  0.0203],
        ...,
        [-0.0205,  0.0164, -0.0105,  ..., -0.0155,  0.0018, -0.0116],
        [-0.0189,  0.0157,  0.0153,  ..., -0.0129, -0.0067, -0.0168],
        [-0.0019,  0.0022, -0.0098,  ...,  0.0199,  0.0004,  0.0170]],
       device='cuda:0')

In [40]:
blubb.model.fc[0].weight

Parameter containing:
tensor([[ 0.0072,  0.0071, -0.0101,  ...,  0.0065,  0.0051,  0.0175],
        [-0.0102,  0.0059,  0.0228,  ..., -0.0115,  0.0006, -0.0125],
        [ 0.0167, -0.0050,  0.0069,  ..., -0.0075, -0.0214,  0.0203],
        ...,
        [-0.0205,  0.0164, -0.0105,  ..., -0.0155,  0.0018, -0.0116],
        [-0.0189,  0.0157,  0.0153,  ..., -0.0129, -0.0067, -0.0168],
        [-0.0019,  0.0022, -0.0098,  ...,  0.0199,  0.0004,  0.0170]],
       device='cuda:0')

In [109]:
model.model.forward(x.float().cuda())

tensor([[-6.3084],
        [-6.4678],
        [-5.9596],
        [-6.4614]], device='cuda:0')

In [110]:
blubb.model.forward(x.float().cuda())

tensor([[-2.0184],
        [-3.5679],
        [-2.5764],
        [-3.3453]], device='cuda:0')

In [87]:
torch.save(blubb.state_dict(),'/tmp/weird1.pth')
torch.save(model.state_dict(),'/tmp/weird2.pth')

In [106]:
model.freeze()
blubb.freeze()
for key in model.state_dict().keys():
    print(key)
    print(torch.sum(blubb.state_dict()[key] - model.state_dict()[key]))

model.conv1.weight
tensor(0., device='cuda:0')
model.bn1.weight
tensor(0., device='cuda:0')
model.bn1.bias
tensor(0., device='cuda:0')
model.bn1.running_mean
tensor(0.0337, device='cuda:0')
model.bn1.running_var
tensor(-2.8077, device='cuda:0')
model.bn1.num_batches_tracked
tensor(-302, device='cuda:0')
model.layer1.0.conv1.weight
tensor(0., device='cuda:0')
model.layer1.0.bn1.weight
tensor(0., device='cuda:0')
model.layer1.0.bn1.bias
tensor(0., device='cuda:0')
model.layer1.0.bn1.running_mean
tensor(2.1207, device='cuda:0')
model.layer1.0.bn1.running_var
tensor(-0.0685, device='cuda:0')
model.layer1.0.bn1.num_batches_tracked
tensor(-302, device='cuda:0')
model.layer1.0.conv2.weight
tensor(0., device='cuda:0')
model.layer1.0.bn2.weight
tensor(0., device='cuda:0')
model.layer1.0.bn2.bias
tensor(0., device='cuda:0')
model.layer1.0.bn2.running_mean
tensor(-0.2326, device='cuda:0')
model.layer1.0.bn2.running_var
tensor(0.0015, device='cuda:0')
model.layer1.0.bn2.num_batches_tracked
tensor(

In [72]:
do_the_eval(model)

HR: 0.888361
LR: 0.528947


In [73]:
do_the_eval(blubb)

HR: 0.565321
LR: 0.913158


In [33]:
for p in model.val_dataloader()[0]:
    if p[3][0] == 'lr':
        print(p[3])

('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')
('lr', 'lr', 'lr', 'lr')


KeyboardInterrupt: 

In [12]:
cm.penalty(model)*100000000

ewcloss: 0.000000


tensor(0., device='cuda:0', grad_fn=<MulBackward0>)

In [4]:
?? nn.functional.bce

Init signature:
 nn.BCEWithLogitsLoss(
    weight=None,
    size_average=None,
    reduce=None,
    reduction='mean',
    pos_weight=None,
)
Source:        
class BCEWithLogitsLoss(_Loss):
    r"""This loss combines a `Sigmoid` layer and the `BCELoss` in one single
    class. This version is more numerically stable than using a plain `Sigmoid`
    followed by a `BCELoss` as, by combining the operations into one layer,
    we take advantage of the log-sum-exp trick for numerical stability.

    The unreduced (i.e. with :attr:`reduction` set to ``'none'``) loss can be described as:

    .. math::
        \ell(x, y) = L = \{l_1,\dots,l_N\}^\top, \quad
        l_n = - w_n \left[ y_n \cdot \log \sigma(x_n)
        + (1 - y_n) \cdot \log (1 - \sigma(x_n)) \right],

    where :math:`N` is the batch size. If :attr:`reduction` is not ``'none'``
    (default ``'mean'``), then

    .. math::
        \ell(x, y) = \begin{cases}
            \operatorname{mean}(L), & \text{if reduction} = \text{'mean

In [6]:
cm.penalty(model)

tensor(0., device='cuda:0', grad_fn=<AddBackward0>)

In [17]:
from torch.autograd import Variable


In [33]:
blubb = {'a': 1, 'b': 2}

In [36]:
blubb.pop('a')

1

In [37]:
blubb

{'b': 2}

In [34]:
a[0].float().cuda()[0].dtype

torch.float32

In [8]:
for x,y in model.train_dataloader():
    print(x)

[tensor([[[[0.0000, 0.0077, 0.0098,  ..., 0.0098, 0.0211, 0.0197],
          [0.0028, 0.0014, 0.0077,  ..., 0.0105, 0.0176, 0.0162],
          [0.0267, 0.0147, 0.0035,  ..., 0.0133, 0.0197, 0.0239],
          ...,
          [0.0183, 0.0246, 0.0204,  ..., 0.2725, 0.1791, 0.1025],
          [0.0091, 0.0056, 0.0028,  ..., 0.0878, 0.0618, 0.0337],
          [0.0154, 0.0077, 0.0063,  ..., 0.0288, 0.0302, 0.0232]],

         [[0.0000, 0.0077, 0.0098,  ..., 0.0098, 0.0211, 0.0197],
          [0.0028, 0.0014, 0.0077,  ..., 0.0105, 0.0176, 0.0162],
          [0.0267, 0.0147, 0.0035,  ..., 0.0133, 0.0197, 0.0239],
          ...,
          [0.0183, 0.0246, 0.0204,  ..., 0.2725, 0.1791, 0.1025],
          [0.0091, 0.0056, 0.0028,  ..., 0.0878, 0.0618, 0.0337],
          [0.0154, 0.0077, 0.0063,  ..., 0.0288, 0.0302, 0.0232]],

         [[0.0000, 0.0077, 0.0098,  ..., 0.0098, 0.0211, 0.0197],
          [0.0028, 0.0014, 0.0077,  ..., 0.0105, 0.0176, 0.0162],
          [0.0267, 0.0147, 0.0035,  ..., 

KeyboardInterrupt: 